In [ ]:
# Colab 환경에서 kaggle.json 파일 업로드
from google.colab import files
files.upload()

# kaggle.json 파일을 .kaggle 디렉토리로 이동
!mkdir -p ~/.kaggle
!mv kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

Saving kaggle.json to kaggle.json


In [ ]:
!kaggle datasets download -d deewakarchakraborty/portrait-paintings

Dataset URL: https://www.kaggle.com/datasets/deewakarchakraborty/portrait-paintings
License(s): CC0-1.0
 84% 188M/223M [00:00<00:00, 892MB/s] 
100% 223M/223M [00:00<00:00, 745MB/s]


In [ ]:
# 데이터셋 다운로드 및 압축 해제
!kaggle datasets download -d deewakarchakraborty/portrait-paintings
!unzip portrait-paintings.zip -d portrait_paintings

스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
  inflating: portrait_paintings/Images/boris-kustodiev_201.jpg  
  inflating: portrait_paintings/Images/boris-kustodiev_202.jpg  
  inflating: portrait_paintings/Images/boris-kustodiev_203.jpg  
  inflating: portrait_paintings/Images/boris-kustodiev_204.jpg  
  inflating: portrait_paintings/Images/boris-kustodiev_205.jpg  
  inflating: portrait_paintings/Images/boris-kustodiev_206.jpg  
  inflating: portrait_paintings/Images/boris-kustodiev_207.jpg  
  inflating: portrait_paintings/Images/boris-kustodiev_208.jpg  
  inflating: portrait_paintings/Images/boris-kustodiev_209.jpg  
  inflating: portrait_paintings/Images/boris-kustodiev_21.jpg  
  inflating: portrait_paintings/Images/boris-kustodiev_210.jpg  
  inflating: portrait_paintings/Images/boris-kustodiev_211.jpg  
  inflating: portrait_paintings/Images/boris-kustodiev_212.jpg  
  inflating: portrait_paintings/Images/boris-kustodiev_213.jpg  
  inflating: portrait_paintings/Images/boris-kustodiev_

In [ ]:
import os, shutil

# real 데이터셋 폴더 생성
os.makedirs("real", exist_ok=True)

# 명화 이미지들을 real 폴더로 복사
for f in os.listdir("portrait_paintings/Images"):
    shutil.copy(os.path.join("portrait_paintings/Images", f), "real")

In [ ]:
!pip install diffusers transformers accelerate safetensors
!pip install xformers  # 메모리 최적화

from diffusers import StableDiffusionPipeline, EulerAncestralDiscreteScheduler
import torch, os

device = "cuda"

model_id = "runwayml/stable-diffusion-v1-5"  # 또는 stabilityai/stable-diffusion-v1-5

pipe = StableDiffusionPipeline.from_pretrained(model_id, torch_dtype=torch.float16)
pipe.scheduler = EulerAncestralDiscreteScheduler.from_config(pipe.scheduler.config)  # ← 중요
pipe.enable_xformers_memory_efficient_attention()  # xformers 깔았으니 이득
pipe.enable_attention_slicing()
pipe = pipe.to(device)

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 117.2/117.2 MB 8.6 MB/s eta 0:00:00


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


model_index.json:   0%|          | 0.00/541 [00:00<?, ?B/s]

Fetching 15 files:   0%|          | 0/15 [00:00<?, ?it/s]

config.json: 0.00B [00:00, ?B/s]

...

config.json:   0%|          | 0.00/743 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/547 [00:00<?, ?B/s]

unet/diffusion_pytorch_model.safetensors:   0%|          | 0.00/3.44G [00:00<?, ?B/s]

vae/diffusion_pytorch_model.safetensors:   0%|          | 0.00/335M [00:00<?, ?B/s]

KeyboardInterrupt: 

In [ ]:
import os
os.makedirs("fake", exist_ok=True)

# 스타일/네거티브 프롬프트
STYLE = ", oil on canvas, visible brush strokes, baroque lighting, chiaroscuro, aged varnish, craquelure, museum quality"
NEG   = "modern digital look, glossy, oversharpen, neon, photo, watermark, text, low detail, blurry, plastic"

prompts = [
    "a classical portrait of a woman, 3/4 view"+STYLE,
    "a renaissance portrait of a man with subtle background"+STYLE,
    "an impressionist portrait with soft edges and visible brushwork"+STYLE,
    "a baroque noble portrait, dramatic contrast"+STYLE,
]

# — steps/CFG/해상도 조정 + 네거티브 + 재현성 seed
g = torch.Generator(device=device)
for i in range(50):
    g.manual_seed(1000 + i)  # 각 샷 다른 seed
    image = pipe(
        prompts[i % len(prompts)],
        negative_prompt=NEG,
        num_inference_steps=36,   # 32~40 권장
        guidance_scale=5.0,       # 4.5~6 권장(높이면 디지털스러워짐)
        width=512, height=512,    # 데이터셋 비율에 맞추면 더 좋음(아래 B-보너스 참고)
        generator=g
    ).images[0]
    image.save(f"fake/fake_{i:05d}.png")

  0%|          | 0/50 [00:00<?, ?it/s]

  0%|          | 0/50 [00:00<?, ?it/s]

  0%|          | 0/50 [00:00<?, ?it/s]

  0%|          | 0/50 [00:00<?, ?it/s]

  0%|          | 0/50 [00:00<?, ?it/s]

...

  0%|          | 0/50 [00:00<?, ?it/s]

  0%|          | 0/50 [00:00<?, ?it/s]

  0%|          | 0/50 [00:00<?, ?it/s]

  0%|          | 0/50 [00:00<?, ?it/s]

  0%|          | 0/50 [00:00<?, ?it/s]

In [ ]:
# img2img로 real 톤 이식
!pip install --quiet diffusers transformers accelerate safetensors xformers

from diffusers import StableDiffusionImg2ImgPipeline, EulerAncestralDiscreteScheduler
from PIL import Image
import random, glob, torch, os

device = "cuda"
model_id = "runwayml/stable-diffusion-v1-5"

pipe_i2i = StableDiffusionImg2ImgPipeline.from_pretrained(model_id, torch_dtype=torch.float16)
pipe_i2i.scheduler = EulerAncestralDiscreteScheduler.from_config(pipe_i2i.scheduler.config)
pipe_i2i.enable_xformers_memory_efficient_attention()
pipe_i2i.enable_attention_slicing()
pipe_i2i = pipe_i2i.to(device)

STYLE = ", oil on canvas, visible brush strokes, baroque lighting, chiaroscuro, aged varnish, craquelure, museum quality"
NEG   = "modern digital look, glossy, oversharpen, neon, photo, watermark, text, low detail, blurry, plastic"

prompts = [
    "a classical portrait of a woman, 3/4 view"+STYLE,
    "a renaissance portrait of a man with subtle background"+STYLE,
    "a baroque noble portrait, dramatic contrast"+STYLE,
    "an impressionist portrait with soft edges"+STYLE,
]

os.makedirs("fake", exist_ok=True)
real_refs = [os.path.join("real", f) for f in os.listdir("real") if f.lower().endswith((".png",".jpg",".jpeg"))]

def _to_mult_of_8(x): return (x // 8) * 8

for i in range(50):
    ref_path = random.choice(real_refs)
    base = Image.open(ref_path).convert("RGB")
    # real 비율 최대한 유지: 긴 변 512, 짧은 변 비율 맞춤 후 8의 배수 보정
    w, h = base.size
    if w >= h:
        new_w, new_h = 512, max(64, int(512 * h / w))
    else:
        new_h, new_w = 512, max(64, int(512 * w / h))
    new_w, new_h = _to_mult_of_8(new_w), _to_mult_of_8(new_h)
    base = base.resize((max(64,new_w), max(64,new_h)), Image.BICUBIC)

    g = torch.Generator(device=device).manual_seed(2000 + i)
    out = pipe_i2i(
        prompts[i % len(prompts)],
        negative_prompt=NEG,
        image=base,
        strength=0.45,            # 0.35~0.55 사이 탐색
        guidance_scale=5.0,       # 4.5~6
        num_inference_steps=36,
        generator=g,
    ).images[0]

    out.save(f"fake/fake_i_{i:05d}.png")

In [ ]:
# A+B: Euler A 샘플러 + STYLE/NEG 그대로 쓰고, img2img에 real 레퍼런스 넣기
!pip install -q diffusers transformers accelerate safetensors xformers

from diffusers import StableDiffusionImg2ImgPipeline, EulerAncestralDiscreteScheduler
from PIL import Image
import torch, os, random, glob

device = "cuda"
model_id = "runwayml/stable-diffusion-v1-5"  # (= 너가 쓰던 모델)

# 파이프라인 (A의 샘플러/옵션)
pipe = StableDiffusionImg2ImgPipeline.from_pretrained(model_id, torch_dtype=torch.float16)
pipe.scheduler = EulerAncestralDiscreteScheduler.from_config(pipe.scheduler.config)  # A: 샘플러 튜닝
pipe.enable_xformers_memory_efficient_attention()
pipe.enable_attention_slicing()
pipe = pipe.to(device)

# A의 프롬프트 튜닝 그대로
STYLE = ", oil on canvas, visible brush strokes, baroque lighting, chiaroscuro, aged varnish, craquelure, museum quality"
NEG   = "modern digital look, glossy, oversharpen, neon, photo, watermark, text, low detail, blurry, plastic"

prompts = [
    "a classical portrait of a woman, 3/4 view"+STYLE,
    "a renaissance portrait of a man with subtle background"+STYLE,
    "an impressionist portrait with soft edges and visible brushwork"+STYLE,
    "a baroque noble portrait, dramatic contrast"+STYLE,
]

# --- real 폴더의 이미지를 레퍼런스로 사용 (B) ---
real_refs = [os.path.join("real", f) for f in os.listdir("real") if f.lower().endswith((".png",".jpg",".jpeg"))]
os.makedirs("fake", exist_ok=True)

def _resize_keep_ratio_to_max512(img: Image.Image):
    w, h = img.size
    if w >= h:
        new_w, new_h = 512, max(64, int(512 * h / w))
    else:
        new_h, new_w = 512, max(64, int(512 * w / h))
    # 8의 배수 보정(디퓨전 아티팩트 감소)
    new_w = (new_w // 8) * 8
    new_h = (new_h // 8) * 8
    new_w = max(64, new_w); new_h = max(64, new_h)
    return img.resize((new_w, new_h), Image.BICUBIC)

# 하이브리드 생성 루프
for i in range(50):
    prompt = prompts[i % len(prompts)]
    ref_path = random.choice(real_refs)  # B: 명화 레퍼런스 한 장 선택
    base = Image.open(ref_path).convert("RGB")
    base = _resize_keep_ratio_to_max512(base)

    g = torch.Generator(device=device).manual_seed(1000 + i)  # 재현성

    # A의 하이퍼파라미터를 유지하면서, B처럼 image=base로 스타일 이식
    img = pipe(
        prompt=prompt,
        negative_prompt=NEG,
        image=base,                 # B: 레퍼런스 이미지 사용
        strength=0.45,              # 0.35~0.55 탐색: 낮을수록 레퍼런스 톤/붓질 보존↑
        guidance_scale=5.0,         # A: 4.5~6 권장
        num_inference_steps=36,     # A: 32~40 권장
        generator=g,
    ).images[0]

    img.save(f"fake/fake_h_{i:05d}.png")

Fetching 15 files:   0%|          | 0/15 [00:00<?, ?it/s]

safety_checker/model.safetensors:   0%|          | 0.00/1.22G [00:00<?, ?B/s]

vae/diffusion_pytorch_model.safetensors:   0%|          | 0.00/335M [00:00<?, ?B/s]

unet/diffusion_pytorch_model.safetensors:   0%|          | 0.00/3.44G [00:00<?, ?B/s]

text_encoder/model.safetensors:   0%|          | 0.00/492M [00:00<?, ?B/s]

...

  0%|          | 0/16 [00:00<?, ?it/s]

  0%|          | 0/16 [00:00<?, ?it/s]

  0%|          | 0/16 [00:00<?, ?it/s]

  0%|          | 0/16 [00:00<?, ?it/s]

  0%|          | 0/16 [00:00<?, ?it/s]